## Preprocess for MEADTTS Dataset

### Config & Utility Functions

In [1]:
import os
import pandas as pd
import numpy as np

RANDOM_SEED = 42

PROJECT_PATH = os.path.join('/', *os.getcwd().split(os.sep)[:-1])

# MEADTTS data path, e.g. data/MEADTTS
MEADTTS_PATH = f'{PROJECT_PATH}/data/MEADTTS'
# MEADTTS audio data path, e.g. data/MEADTTS/raw
MEADTTS_RAW_PATH = f'{MEADTTS_PATH}/raw'
# MEADTTS transcript data path, e.g. data/MEADTTS/transcript
MEADTTS_TRANSCRIPT_PATH = f'{MEADTTS_PATH}/mead_transcript'

# Copy and split MEADTTS raw data by speaker folder for MFA, e.g. data/MEAD_MFA
MEADTTS_MFA = f'{PROJECT_PATH}/data/MEADTTS_MFA'

# preprocessed MFA data, e.g. data/MEADTTS_MFA_preprocessed
MEADTTS_MFA_PREPRO = f'{PROJECT_PATH}/data/MEADTTS_MFA_preprocessed'

# filelist path for load MEADTTS data for training, validation and testing, e.g. EMITTS/filelist/MEADTTS
FILELIST_PATH = f'{PROJECT_PATH}/EMITTS/filelist/MEADTTS'
os.makedirs(FILELIST_PATH, exist_ok=True)

# EMO_FEATURE_SAVE_PATH is the path to save the extracted emotion features, e.g. EPAlign/mmefeature/MEADTTS
EMO_FEATURE_SAVE_PATH = f"{PROJECT_PATH}/EPAlign/mmefeature/MEADTTS"

# find all the wav files
def find_file(path, suffix):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if name.endswith(suffix):
                result.append(os.path.join(root, name))
    return result

def load_filelist(filename, split="|"):
    with open(filename, encoding='utf-8') as f:
        filelist = [line.strip().split(split) for line in f]
    return filelist

### Split Data into Train, Validation, and Test

In [ ]:
wav_files = find_file(MEADTTS_RAW_PATH, '.wav')
wav_dict = {}
for wav_file in wav_files:
    wav_dict[wav_file.split('/')[-1][:-4]] = wav_file

assert len(wav_dict) == 31_055

df_total = pd.DataFrame(columns=['filename', 'text', 'emotiontag'])
for file in find_file(MEADTTS_TRANSCRIPT_PATH, '.txt'):
    filename = file.split('/')[-1][:-4]
    text = open(file, 'r', encoding='utf-8').read().strip()
    emotion_tag = f'{filename.split("_")[1]}_{filename.split("_")[2]}_{filename.split("_")[3]}'
    df_total = pd.concat([df_total, pd.DataFrame({'filename': [filename], 'text': [text], 'emotiontag': [emotion_tag]})], ignore_index=True)

emotion_tags_set = set(df_total['emotiontag'])
emotion_tags = [emotion_tag for emotion_tag in emotion_tags_set]
train_df = pd.DataFrame(columns=['filename', 'text', 'emotiontag'])
val_df = pd.DataFrame(columns=['filename', 'text', 'emotiontag'])
test_df = pd.DataFrame(columns=['filename', 'text', 'emotiontag'])

for emotion_tag in emotion_tags:
    df_e = df_total[df_total['emotiontag'] == emotion_tag].to_numpy()
    # split the data into train, val, test with 80%, 10%, 10%
    np.random.seed(42)
    np.random.shuffle(df_e)
    df_tr = pd.DataFrame(df_e[:int(len(df_e)*0.8)], columns=['filename', 'text', 'emotiontag'])
    df_va = pd.DataFrame(df_e[int(len(df_e)*0.8):int(len(df_e)*0.9)], columns=['filename', 'text', 'emotiontag'])
    df_te = pd.DataFrame(df_e[int(len(df_e)*0.9):], columns=['filename', 'text', 'emotiontag'])
    train_df = pd.concat([train_df, df_tr])
    val_df = pd.concat([val_df, df_va])
    test_df = pd.concat([test_df, df_te])

assert len(train_df) == 24_836
assert len(val_df) == 3_103
assert len(test_df) == 3_116

# save train, val, test filelist
train_val_test = ["train", "val", "test"]
dfs = [train_df, val_df, test_df]
for i, df in enumerate(dfs):
    new_filelist = f'{FILELIST_PATH}/MEADTTS_audio_sid_text_efeature_{train_val_test[i]}_filelist.txt'
    with open(new_filelist, 'w', encoding='utf-8') as f:
        for index, row in df.iterrows():
            wav_path = wav_dict[row['filename']]
            sid = int(row['filename'].split('_')[0][1:]) if row['filename'].split('_')[0][0] == 'W' else int(row['filename'].split('_')[0][1:]) + 40
            text = row['text']
            mmefeature_path = f'{EMO_FEATURE_SAVE_PATH}/{row['emotiontag'].lower()}.pt'
            f.write(f'{wav_path}|{str(sid)}|{text}|{mmefeature_path}\n')
    print(f'Saved {new_filelist}')

### Clean Text for VITS Variance

In [ ]:
import sys
sys.path.append(f'{PROJECT_PATH}/EMITTS/VITS')
import utils.text_utils as text

# test_text = test_df['text'].to_list()
# train_text = train_df['text'].to_list()

# t = text._clean_text(test_text, ["english_cleaners2"])

# save train, val, test clean text filelist
train_val_test = ["train", "val", "test"]
dfs = [train_df, val_df, test_df]
for i, df in enumerate(dfs):
    new_filelist = f'{FILELIST_PATH}/MEADTTS_audio_sid_text_efeature_{train_val_test[i]}_filelist.txt.cleaned'
    clean_texts = text._clean_text(df['text'].to_list(), ["english_cleaners2"])
    with open(new_filelist, 'w', encoding='utf-8') as f:
        for index, row in df.iterrows():
            wav_path = wav_dict[row['filename']]
            sid = int(row['filename'].split('_')[0][1:]) if row['filename'].split('_')[0][0] == 'W' else int(row['filename'].split('_')[0][1:]) + 40
            clean_text = clean_texts[index]
            mmefeature_path = f'{EMO_FEATURE_SAVE_PATH}/{row['emotiontag'].lower()}.pt'
            f.write(f'{wav_path}|{str(sid)}|{clean_text}|{mmefeature_path}\n')
    print(f'Saved {new_filelist}')

### Clean Text for FastSpeech2 Variance

[Montreal Forced Aligner](https://montreal-forced-aligner.readthedocs.io/en/latest/) (MFA) is used to obtain the alignments between the utterances and the phoneme sequences. 

The following steps are used to align the utterances and the phoneme sequences refer to the [mfa-ljspeech.ipynb](https://gist.github.com/12264d15afad861cb897f7a20a01762e.git):

1. install using [mamba](https://mamba.readthedocs.io/en/latest/installation/mamba-installation.html#)
    ```bash
    # create and install
    mamba create -n aligner -c conda-forge montreal-forced-aligner

    # install 
    mamba install -c conda-forge montreal-forced-aligner
    ```

2. download and modify the pre-trained model and the lexicon (current working directory is the root of the project)
    ```bash
    mamba activate aligner
    wget -O ./data/english.zip https://github.com/MontrealCorpusTools/mfa-models/raw/main/acoustic/english.zip
    wget -O ./data/librispeech-lexicon.txt http://www.openslr.org/resources/11/librispeech-lexicon.txt
    ```

In [4]:
# see: https://github.com/MontrealCorpusTools/Montreal-Forced-Aligner/pull/480
import re
lexicon = open(f"{PROJECT_PATH}/data/librispeech-lexicon.txt").readlines()
sp = re.compile(r"\s+")
with open(f"{PROJECT_PATH}/data/modified_lexicon.txt", "w") as f:
    for line in lexicon:
        word, *phonemes = sp.split(line.strip())
        phonemes = " ".join(phonemes)
        f.write(f"{word}\t{phonemes}\n")

3. prepare MEADTTS_SP data (split the data with speaker id)

In [ ]:
os.makedirs(MEADTTS_MFA, exist_ok=True)

speakers = [speaker for speaker in set(df_total['filename'].apply(lambda x: x.split('_')[0]))]
for speaker in speakers:
    os.system(f'mkdir -p {MEADTTS_MFA}/{speaker}')

wav_paths = find_file(MEADTTS_RAW_PATH, '.wav')
for wav_path in wav_paths:
    speaker = wav_path.split('/')[-1].split('_')[0]
    os.system(f'cp {wav_path} {MEADTTS_MFA}/{speaker}')
print(f'Copied all .wav files from {MEADTTS_RAW_PATH} to {MEADTTS_MFA} by speaker folder')

for file in find_file(MEADTTS_TRANSCRIPT_PATH, '.txt'):
    # print(file)
    filename = file.split('/')[-1][:-4]
    os.system(f'cp {file} {MEADTTS_MFA}/{filename.split("_")[0]}')

4. using MFA to align the utterances and the phoneme sequences (current working directory is the root of the project)
    ```bash
    mfa align -t ./data/mfa_temp -j 47 ./data/MEADTTS_MFA ./data/modified_lexicon.txt ./data/english.zip ./data/MEADTTS_MFA
    ```

In [ ]:
for speaker in speakers:
    os.system(f'mv {MEADTTS_MFA}/{speaker}/* {MEADTTS_MFA}')
    os.system(f'rm -r {MEADTTS_MFA}/{speaker}')
    print(f'Moved all files from {MEADTTS_MFA}/{speaker} to {MEADTTS_MFA}')

In [ ]:
import sys
sys.path.append(PROJECT_PATH)
from EMITTS.FastSpeech2.config.config import TrainConfig
sys.path.append(f'{PROJECT_PATH}/EMITTS/FastSpeech2')
from EMITTS.FastSpeech2.src.preprocess.preprocess import Preprocessor
cfg = TrainConfig()

cfg.raw_data_path = MEADTTS_MFA
cfg.train_ids_path = f'{FILELIST_PATH}/MEADTTS_audio_sid_text_efeature_train_filelist.txt'
cfg.val_ids_path = f'{FILELIST_PATH}/MEADTTS_audio_sid_text_efeature_val_filelist.txt'
cfg.test_ids_path = f'{FILELIST_PATH}/MEADTTS_audio_sid_text_efeature_test_filelist.txt'
cfg.preprocessed_data_path = MEADTTS_MFA_PREPRO

preprocessor = Preprocessor(cfg)
preprocessor.run()